In [ ]:

# def squareform_sp(w):
#     """
#     Sparse counterpart of numpy's squareform.

#     Parameters:
#     -----------
#     w : 
#         - A sparse vector with n(n-1)/2 elements, or
#         - A sparse matrix of size [n, n] with a zero diagonal,
#         - or a dense array, in which case we check its sparsity.

#     Returns:
#     --------
#     W : 
#         - If input w is a vector, returns W: an n-by-n sparse symmetric matrix.
#         - If input w is a matrix, returns w: an n(n-1)/2-by-1 sparse vector.
#     """

#     # CHANGED: If w is not sparse, use np.count_nonzero(w) instead of w.nnz
#     if not sp.issparse(w):
#         dense_nnz = np.count_nonzero(w)
#         if dense_nnz / w.size > 0.1:
#             raise ValueError("Use standard squareform for non-sparse vector!")
#     else:
#         # w is sparse
#         if w.nnz / w.size > 0.1:
#             raise ValueError("Use standard squareform for non-sparse vector!")

#     # Check if input is a vector (one dimension is 1)
#     if w.shape[0] == 1 or w.shape[1] == 1:
#         # VECTOR -> MATRIX
#         l = w.shape[0] if w.shape[1] == 1 else w.shape[1]
#         n = int(round((1 + np.sqrt(1 + 8*l)) / 2))
        
#         # Check input size
#         if l != n*(n-1)//2:
#             raise ValueError("Bad vector size!")

#         # Extract nonzeros from vector
#         if w.shape[0] == 1:
#             ind_vec = w.nonzero()[1]
#         else:
#             ind_vec = w.nonzero()[0]
#         s = w.data if sp.issparse(w) else w[ind_vec]  # CHANGED: if dense, directly index
#         num_nz = len(ind_vec)

#         # Compute (ind_i, ind_j) as in MATLAB
#         ind_i = np.zeros(num_nz, dtype=int)
#         ind_j = np.zeros(num_nz, dtype=int)

#         curr_row = 0
#         offset = 0
#         length = n - 1

#         for ii in range(num_nz):
#             ind_vec_i_1b = ind_vec[ii] + 1
#             while ind_vec_i_1b > (length + offset):
#                 offset += length
#                 length -= 1
#                 curr_row += 1

#             ind_i[ii] = curr_row
#             ind_j[ii] = (ind_vec_i_1b - offset) + (n - length) - 1

#         # Construct symmetric matrix: upper + lower
#         W = sp.csr_matrix((np.concatenate([s, s]),
#                            (np.concatenate([ind_i, ind_j]),
#                             np.concatenate([ind_j, ind_i]))),
#                           shape=(n, n))
#         return W

#     else:
#         # MATRIX -> VECTOR
#         m, n = w.shape
#         if m != n or not np.allclose(np.diag(w), 0):
#             raise ValueError("Matrix has to be square with zero diagonal!")

#         ind_i, ind_j = w.nonzero() if sp.issparse(w) else np.nonzero(w)
#         s = w.data if sp.issparse(w) else w[ind_i, ind_j]

#         # Keep only upper triangular part
#         ind_upper = ind_i < ind_j
#         ind_i = ind_i[ind_upper]
#         ind_j = ind_j[ind_upper]
#         s = s[ind_upper]

#         # Convert (i,j) to vector index (0-based)
#         new_ind = (n * ind_i - (ind_i*(ind_i+1))//2 + ind_j - ind_i - 1).astype(int)

#         return sp.csr_matrix((s, (new_ind, np.zeros_like(new_ind))),
#                              shape=(n*(n-1)//2, 1))

# def squareform_sp(w):
#     """
#     Parameters
#     ----------
#     w : array-like or sparse matrix
#         If w is a vector of length n*(n-1)/2, it returns an n-by-n symmetric
#         sparse matrix with zero diagonal.
#         If w is an n-by-n sparse matrix with zero diagonal, returns a sparse
#         vector of length n*(n-1)/2 representing the upper-triangular part.

#     Returns
#     -------
#     w_out : sparse matrix or sparse vector
#         If input was vector, output is an n-by-n sparse matrix.
#         If input was matrix, output is a sparse vector.
#     """

#     # Convert input to a known sparse type if not already
#     if not isspmatrix(w) and isinstance(w, np.ndarray):
#         density = np.count_nonzero(w)/w.size if w.size > 0 else 0
#         if density > 1/10:
#             # If too dense, fallback to dense squareform
#             return squareform(w)
#         else:
#             w = csr_matrix(w)
#     elif not isspmatrix(w):
#         # Convert lists or other array-likes to sparse
#         arr = np.array(w)
#         if arr.ndim == 1:
#             w = csr_matrix(arr.reshape(-1, 1))
#         else:
#             w = csr_matrix(arr)

#     # Distinguish between vector and matrix based on shape
#     if w.ndim == 2 and w.shape[1] != 1:
#         # Matrix -> Vector
#         m, n = w.shape
#         if m != n:
#             raise ValueError('Matrix must be square!')
#         diag_vals = w.diagonal()
#         if not np.allclose(diag_vals, 0):
#             raise ValueError('Matrix diagonal must be zero!')

#         ind_i, ind_j, s = find(w)
#         # Keep only upper triangular entries
#         mask_upper = ind_i < ind_j
#         ind_i = ind_i[mask_upper]
#         ind_j = ind_j[mask_upper]
#         s = s[mask_upper]

#         # Convert (i,j) to condensed index k using zero-based formula:
#         # k = i*(n - 1) - (i*(i-1))//2 + (j - i - 1)
#         new_ind = ind_i*(n-1) - (ind_i*(ind_i-1))//2 + (ind_j - ind_i - 1)
#         length_vec = n*(n-1)//2
#         if new_ind.max() >= length_vec or new_ind.min() < 0:
#             raise ValueError("Indexing error: computed condensed index out of range.")

#         # Create sparse vector
#         w_out = csr_matrix((s, (new_ind, np.zeros_like(new_ind))), shape=(length_vec, 1))

#     else:
#         # Vector -> Matrix
#         # w is a vector (length_vec x 1)
#         if w.shape[1] != 1:
#             raise ValueError("Vector input should be a single column sparse vector.")
#         l = w.shape[0]
#         n = int(round((1 + sqrt(1+8*l))/2))
#         if l != n*(n-1)//2:
#             raise ValueError('Bad vector size!')

#         ind_vec, _, s = find(w)
#         num_nz = len(ind_vec)
#         ind_i = np.zeros(num_nz, dtype=int)
#         ind_j = np.zeros(num_nz, dtype=int)

#         # Inverse mapping: given k in [0, ..., l-1], find (i,j)
#         # We use the loop logic given in the original MATLAB code
#         curr_row = 1
#         offset = 0
#         length_line = n - 1
#         for idx in range(num_nz):
#             ind_val = ind_vec[idx] + 1  # Convert to 1-based index
#             while ind_val > (length_line + offset):
#                 offset += length_line
#                 length_line -= 1
#                 curr_row += 1
#             # Now curr_row indicates the row i (1-based)
#             # The corresponding column j is ind_val - offset + (n - length_line)
#             # Convert both to zero-based indexing:
#             i_zero = curr_row - 1
#             j_zero = ind_val - offset + (n - length_line) - 1
#             ind_i[idx] = i_zero
#             ind_j[idx] = j_zero

#         # Construct the symmetric matrix
#         row_inds = np.concatenate((ind_i, ind_j))
#         col_inds = np.concatenate((ind_j, ind_i))
#         data = np.concatenate((s, s))
#         w_out = csr_matrix((data, (row_inds, col_inds)), shape=(n, n))

#     return w_out


In [ ]:
# def squareform_sp(w):
#     """
#     Sparse counterpart of numpy's squareform

#     Parameters:
#     w : sparse vector with n(n-1)/2 elements OR matrix with size [n, n] and zero diagonal

#     Returns:
#     W : matrix form of input vector w OR vector form of input matrix W
#     """

#     # If input is not sparse, it doesn't make sense to use this function
#     if not sp.issparse(w) and w.nnz / w.size > 1/10:
#         raise ValueError("Use standard squareform for non-sparse vector!")

#     if w.shape[0] == 1 or w.shape[1] == 1:  # It's a vector
#         # VECTOR -> MATRIX
#         l = w.shape[0] if w.shape[1] == 1 else w.shape[1]
#         n = int(round((1 + np.sqrt(1 + 8*l)) / 2))

#         # Check input
#         if l != n*(n-1)//2:
#             raise ValueError("Bad vector size!")

#         ind_vec, s = w.nonzero()[1] if w.shape[0] == 1 else w.nonzero()[0], w.data
#         num_nz = len(ind_vec)

#         # Indices inside the matrix
#         ind_i, ind_j = np.zeros(num_nz, dtype=int), np.zeros(num_nz, dtype=int)

#         k = 0
#         for i in range(n-1):
#             for j in range(i+1, n):
#                 if k < num_nz and ind_vec[k] == k:
#                     ind_i[k] = i
#                     ind_j[k] = j
#                     k += 1

#         # For the lower triangular part just add the transposed matrix
#         return sp.csr_matrix((np.concatenate([s, s]),
#                               (np.concatenate([ind_i, ind_j]), np.concatenate([ind_j, ind_i]))),
#                              shape=(n, n))

#     else:  # It's a matrix
#         # MATRIX -> VECTOR
#         m, n = w.shape
#         if m != n or not np.all(w.diagonal() == 0):
#             raise ValueError("Matrix has to be square with zero diagonal!")

#         ind_i, ind_j = w.nonzero()
#         s = w.data

#         # Keep only upper triangular part
#         ind_upper = ind_i < ind_j
#         ind_i, ind_j, s = ind_i[ind_upper], ind_j[ind_upper], s[ind_upper]

#         # Compute new (vector) index from (i,j) (matrix) indices
#         new_ind = (n*ind_i - ind_i*(ind_i+1)//2 + ind_j - ind_i - 1).astype(int)

#         return sp.csr_matrix((s, (new_ind, np.zeros_like(new_ind))),
#                              shape=(n*(n-1)//2, 1))

def squareform_sp(w):
    """
    Sparse counterpart of numpy's squareform.

    Parameters:
    -----------
    w : 
        - A sparse vector with n(n-1)/2 elements, or
        - A sparse matrix of size [n, n] with a zero diagonal.

    Returns:
    --------
    W : 
        - If input w is a vector, returns W: an n-by-n sparse symmetric matrix.
        - If input w is a matrix, returns w: an n(n-1)/2-by-1 sparse vector.
    """

    # If input is not sparse and relatively dense, advise using standard squareform
    if not sp.issparse(w) and (w.nnz / w.size > 0.1):
        raise ValueError("Use standard squareform for non-sparse vector!")

    # Check if input is a vector (one dimension is 1)
    if w.shape[0] == 1 or w.shape[1] == 1:
        # VECTOR -> MATRIX
        l = w.shape[0] if w.shape[1] == 1 else w.shape[1]
        n = int(round((1 + np.sqrt(1 + 8*l)) / 2))
        
        # Check input size
        if l != n*(n-1)//2:
            raise ValueError("Bad vector size!")

        # Extract nonzeros from vector
        if w.shape[0] == 1:
            ind_vec = w.nonzero()[1]
        else:
            ind_vec = w.nonzero()[0]
        s = w.data
        num_nz = len(ind_vec)

        # Replicate MATLAB logic to find (ind_i, ind_j)
        ind_i = np.zeros(num_nz, dtype=int)
        ind_j = np.zeros(num_nz, dtype=int)

        curr_row = 0
        offset = 0
        length = n - 1

        # MATLAB code uses 1-based indexing; adapt by adding 1 to ind_vec
        for ii in range(num_nz):
            ind_vec_i_1b = ind_vec[ii] + 1  # convert to 1-based index
            while ind_vec_i_1b > (length + offset):
                offset += length
                length -= 1
                curr_row += 1

            # Now map back to zero-based indices
            # MATLAB: ind_i(ii) = curr_row
            #         ind_j(ii) = ind_vec_i - offset + (n-len)
            # For zero-based indexing in Python:
            # ind_i(ii) is already zero-based (curr_row starts from 0)
            # ind_j(ii) = (ind_vec_i_1b - offset) + (n - length) - 1 to zero-base it
            ind_i[ii] = curr_row
            ind_j[ii] = (ind_vec_i_1b - offset) + (n - length) - 1

        # Construct symmetric matrix: upper + lower
        W = sp.csr_matrix((np.concatenate([s, s]),
                           (np.concatenate([ind_i, ind_j]),
                            np.concatenate([ind_j, ind_i]))),
                          shape=(n, n))
        return W

    else:
        # MATRIX -> VECTOR
        m, n = w.shape
        if m != n or not np.all(w.diagonal() == 0):
            raise ValueError("Matrix has to be square with zero diagonal!")

        ind_i, ind_j = w.nonzero()
        s = w.data

        # Keep only upper triangular part
        ind_upper = ind_i < ind_j
        ind_i = ind_i[ind_upper]
        ind_j = ind_j[ind_upper]
        s = s[ind_upper]

        # Convert (i,j) to vector index:
        # In MATLAB:
        # new_ind = ind_j + (ind_i-1)*n - ind_i.*(ind_i+1)/2;
        # This is for 1-based indexing. For zero-based:
        # new_ind = (n*ind_i) - (ind_i*(ind_i+1))//2 + (ind_j - ind_i - 1)
        new_ind = (n * ind_i - (ind_i*(ind_i+1))//2 + ind_j - ind_i - 1).astype(int)

        return sp.csr_matrix((s, (new_ind, np.zeros_like(new_ind))),
                             shape=(n*(n-1)//2, 1))
        
        
        
        

import numpy as np
from scipy import sparse as sp

def squareform_sp(w):
    """
    Sparse counterpart of NumPy's squareform.

    Parameters:
    -----------
    w :
        - A sparse vector with n(n-1)/2 elements, or
        - A sparse matrix of size [n, n] with a zero diagonal.

    Returns:
    --------
    W :
        - If input w is a vector, returns W: an n-by-n sparse symmetric matrix.
        - If input w is a matrix, returns w: an n(n-1)/2-by-1 sparse vector.
    """
    if not sp.isspmatrix(w) and not isinstance(w, np.ndarray):
        raise TypeError("Input must be a sparse matrix or a NumPy array.")

    # Handle dense input by converting to sparse
    if isinstance(w, np.ndarray):
        density = np.count_nonzero(w) / w.size if w.size > 0 else 0
        if density > 0.1:
            raise ValueError("Use NumPy's squareform for dense vectors or matrices.")
        w = sp.csr_matrix(w)

    if w.shape[0] == 1 or w.shape[1] == 1:  # VECTOR -> MATRIX
        l = w.shape[0] if w.shape[1] == 1 else w.shape[1]
        n = int(round((1 + np.sqrt(1 + 8 * l)) / 2))

        # Check input size
        if l != n * (n - 1) // 2:
            raise ValueError("Invalid vector size for conversion.")

        ind_vec = w.nonzero()[1] if w.shape[0] == 1 else w.nonzero()[0]
        s = w.data
        num_nz = len(ind_vec)

        ind_i = np.zeros(num_nz, dtype=int)
        ind_j = np.zeros(num_nz, dtype=int)

        curr_row = 0
        offset = 0
        length = n - 1

        for ii in range(num_nz):
            ind_vec_i_1b = ind_vec[ii] + 1  # Convert to 1-based index
            while ind_vec_i_1b > (length + offset):
                offset += length
                length -= 1
                curr_row += 1

            ind_i[ii] = curr_row
            ind_j[ii] = (ind_vec_i_1b - offset) + (n - length) - 1

        # Construct symmetric matrix: upper + lower
        W = sp.csr_matrix((np.concatenate([s, s]),
                           (np.concatenate([ind_i, ind_j]),
                            np.concatenate([ind_j, ind_i]))),
                          shape=(n, n))
        return W

    else:  # MATRIX -> VECTOR
        m, n = w.shape
        if m != n or not np.allclose(w.diagonal(), 0):
            raise ValueError("Input matrix must be square with a zero diagonal.")

        ind_i, ind_j = w.nonzero()
        s = w.data

        # Keep only upper triangular part
        ind_upper = ind_i < ind_j
        ind_i = ind_i[ind_upper]
        ind_j = ind_j[ind_upper]
        s = s[ind_upper]

        # Convert (i, j) to vector index
        new_ind = (n * ind_i - (ind_i * (ind_i + 1)) // 2 + ind_j - ind_i - 1).astype(int)

        return sp.csr_matrix((s, (new_ind, np.zeros_like(new_ind))),
                             shape=(n * (n - 1) // 2, 1))



# def sum_squareform(n, mask=None):
#     """
#     Computes the sum and transpose sum matrices in a squareform format.

#     Parameters:
#     -----------
#     n : int
#         The size of the squareform matrix.

#     mask : array-like, optional
#         A mask to filter the indices. The length of the mask must be n(n-1)/2.
#         If provided, only the elements corresponding to the non-zero values in
#         the mask are considered.

#         Returns:
#     --------
#     S : csr_matrix
#         A sparse matrix of shape (n, n_cols), where n_cols = n(n-1)/2 if no mask is given.
#         When a mask is provided, n_cols = nnz(mask).

#     St : csr_matrix
#         A sparse matrix of shape (n_cols, n), where n_cols = n(n-1)/2 if no mask is given,
#         or n_cols = nnz(mask) if a mask is provided.

#     Raises:
#     -------
#     ValueError
#         If the length of the mask is not equal to n(n-1)/2.
#     """
#     if mask is not None:
#         mask = np.asarray(mask).flatten()
#         if len(mask) != n * (n - 1) // 2:
#             raise ValueError('Mask size has to be n(n-1)/2')

#         ind_vec = np.flatnonzero(mask)
#         ncols = len(ind_vec)

#         I = np.zeros(ncols, dtype=int)
#         J = np.zeros(ncols, dtype=int)

#         curr_row = 0
#         offset = 0
#         len_row = n - 1
#         for ii in range(ncols):
#             ind_vec_i = ind_vec[ii]
#             while ind_vec_i >= (len_row + offset):
#                 offset += len_row
#                 len_row -= 1
#                 curr_row += 1
#             I[ii] = curr_row
#             J[ii] = ind_vec_i - offset + curr_row + 1

#     else:
#         ncols = (n * (n - 1)) // 2
#         I = np.zeros(ncols, dtype=int)
#         J = np.zeros(ncols, dtype=int)

#         k = 0
#         for i in range(n):
#             for j in range(i + 1, n):
#                 I[k] = i
#                 J[k] = j
#                 k += 1

#     St = csr_matrix((np.ones(ncols), (np.arange(ncols), I)), shape=(ncols, n))
#     S = csr_matrix((np.ones(ncols), (np.arange(ncols), J)), shape=(ncols, n)).T + St.T

#     return S, St


In [ ]:

# def prox_sum_log(x, gamma, param=None):
#     """
#     Proximal operator of log-barrier - sum(log(x))

#     Parameters:
#     x : Input signal (vector or matrix)
#     gamma : Regularization parameter
#     param : Dictionary of optional parameters

#     Returns:
#     sol : Solution
#     info : Dictionary summarizing information at convergence
#     """
#     if param is None:
#         param = {}

#     verbose = param.get('verbose', 1)

#     if gamma < 0:
#         raise ValueError('Gamma cannot be negative')
#     elif gamma == 0:
#         stop_error = True
#     else:
#         stop_error = False

#     t1 = time.time()

#     if stop_error:
#         sol = x
#         info = {
#             'algo': 'prox_sum_log',
#             'iter': 0,
#             'final_eval': 0,
#             'crit': '--',
#             'time': time.time() - t1
#         }
#         return sol, info

#     sol = (x + np.sqrt(x**2 + 4*gamma)) / 2
#     final_eval = -gamma * np.sum(np.log(x[x > 0]))  # Ignore non-positive elements for log

#     info = {
#         'algo': 'prox_sum_log',
#         'iter': 0,
#         'final_eval': final_eval,
#         'crit': '--',
#         'time': time.time() - t1
#     }

#     if verbose >= 1:
#         print(f'  prox_sum_log: - sum(log(x)) = {info["final_eval"] / gamma:.6e}')
#         if verbose > 1:
#             n_neg = np.sum(x <= 0)
#             if n_neg > 0:
#                 print(f' ({n_neg} negative elements, log not defined, check stability)')
#         print()

#     return sol, info

In [ ]:
# import numpy as np
# from scipy.sparse import coo_matrix, csr_matrix

# def sum_squareform(n, mask=None):
#     """
#     Computes the sum and transpose sum matrices in a squareform format, matching MATLAB's sum_squareform.

#     Parameters
#     ----------
#     n : int
#         Number of nodes.
#     mask : array-like, optional
#         A mask of length n(n-1)/2. If provided, only entries with mask[i]!=0 are included.

#     Returns
#     -------
#     S : csr_matrix
#         A sparse matrix of shape (n, n_cols), where n_cols = n(n-1)/2 if no mask is given,
#         or n_cols = nnz(mask) if a mask is provided.
#     St : csr_matrix
#         A sparse matrix of shape (n_cols, n).
#     """
#     if mask is not None:
#         mask = np.asarray(mask).ravel()
#         if len(mask) != n*(n-1)//2:
#             raise ValueError('Mask size has to be n(n-1)/2')

#         # Find nonzero indices in mask
#         ind_vec = np.flatnonzero(mask)  # zero-based indexing
#         ncols = len(ind_vec)

#         # Following MATLAB logic:
#         # curr_row starts at 1 in MATLAB, we will adjust later
#         I = np.zeros(ncols, dtype=int)
#         J = np.zeros(ncols, dtype=int)

#         curr_row = 1
#         offset = 0
#         length = n - 1

#         # The MATLAB code loops from 1 to ncols, we do the same in Python
#         # but remember we are using zero-based indexing for arrays.
#         for ii in range(ncols):
#             ind_vec_i = ind_vec[ii] + 1  # Convert to 1-based indexing for logic
#             while ind_vec_i > (length + offset):
#                 offset += length
#                 length -= 1
#                 curr_row += 1
#             I[ii] = curr_row
#             J[ii] = ind_vec_i - offset + (n - length)
        
#     else:
#         # No mask case:
#         # number of columns = n(n-1)/2
#         ncols = (n * (n - 1)) // 2
#         I = np.zeros(ncols, dtype=int)
#         J = np.zeros(ncols, dtype=int)

#         # Replicate MATLAB code:
#         # In MATLAB:
#         # k = 1;
#         # for i=2:n
#         #   I(k:k+(n-i)) = i:n;
#         #   k = k+(n-i+1);
#         # end
#         # Similarly for J:
#         # k = 1;
#         # for i=2:n
#         #   J(k:k+(n-i)) = i-1;
#         #   k = k+(n-i+1);
#         # end

#         # We must carefully adapt 1-based indexing from MATLAB to 0-based Python.
#         # In MATLAB i ranges 2 to n, and indices start at k=1.
#         # We'll do it in Python with zero-based indexing and then add one if needed.
#         # Actually, since final indexing for S and St must be zero-based, we just
#         # carefully compute directly in zero-based form.

#         # Let's directly compute the pairs (I,J) that would be found in W:
#         # The MATLAB code effectively enumerates upper-triangular entries (excluding diagonal)
#         # row by row.

#         # We'll construct them directly:
#         # Edges are (i,j) with i<j.
#         # The order in MATLAB after rearrangement:
#         # I's are from line: I(k:k+(n-i)) = i:n
#         # but note i runs from 2 to n in MATLAB (1-based).
#         # In zero-based indexing, that would be i runs from 1 to n-1,
#         # I(k:k+(n-(i+1))) = [i+1 ... n-1 in zero-based], which is i+1 to n-1 inclusive
#         # Actually, let's just mimic the logic with a Python loop:

#         pos = 0
#         for i_m in range(2, n+1):  # i_m is MATLAB indexing, from 2 to n
#             i_zero = i_m - 1  # zero-based index for row
#             count = (n - i_m + 1)
#             I[pos:pos+count] = np.arange(i_m, n+1) - 1  # i_m ... n are 1-based, subtract 1 to zero-base
#             pos += count

#         pos = 0
#         for i_m in range(2, n+1):
#             i_zero = i_m - 1
#             count = (n - i_m + 1)
#             J[pos:pos+count] = (i_m - 1) - 1  # i_m-1 is the column index in MATLAB (1-based), subtract 1 more to zero-base
#             # Wait, we must re-check J:
#             # MATLAB: J(k:k+(n-i)) = i-1;
#             # If i_m = 2 (MATLAB), i-1=1. Zero-based that's 0.
#             # Actually i_m-1 gives a zero-based index directly for J.
#             J[pos:pos+count] = (i_m - 1) - 1 + 1
#             # Let's rethink step by step:
#             # In MATLAB: J(k:k+(n-i)) = i-1;
#             # For i=2: J(...) = 1 (1-based indexing)
#             # In zero-based indexing, i=2 => i_zero=1, i-1=1 means zero-based it should be 0 actually?
#             # Wait, MATLAB indexing for J:
#             # If i=2: J(...) = 1 means J is assigned with "1".
#             # In zero-based we want the column index i-1 means i_m-1 as zero-based? Actually, i_m-1 in MATLAB is 1, which is already zero-based index. So J should just be i_m-2 in zero-based:
#             # Because i_m=2 => i_m-2=0 zero-based column.
#             # So J(...) = i_m-2
#             # Let's just do J(...) = i_m-2:
#             J[pos:pos+count] = i_m - 2
#             pos += count

#     # Now we have I,J in 1-based indexing for the mask case, and already zero-based for no-mask.
#     # For the mask case:
#     if mask is not None:
#         # MATLAB indexing was 1-based for I,J. Convert to 0-based.
#         I -= 1
#         J -= 1

#     # According to MATLAB:
#     # St = sparse([1:ncols, 1:ncols], [I, J], 1, ncols, n)
#     # Then S = St'
#     # In Python (0-based), [1:ncols] corresponds to [0, ..., ncols-1].
#     # We'll build St using coo_matrix:
#     row_idx = np.concatenate([np.arange(ncols), np.arange(ncols)])
#     col_idx = np.concatenate([I, J])
#     data = np.ones(2*ncols, dtype=float)

#     St = coo_matrix((data, (row_idx, col_idx)), shape=(ncols, n)).tocsr()

#     # S = St^T
#     S = St.T

#     return S, St


# def sum_squareform(n, mask=None):
#     """
#     Python version of the MATLAB function sum_squareform.
    
#     Parameters:
#     -----------
#     n : int
#         Size of the matrix W (an n-by-n matrix).
#     mask : array-like or None
#         If given, must be of size n(n-1)/2. Non-zero entries indicate
#         which elements in w = squareform(W) are considered.
    
#     Returns:
#     --------
#     S : scipy.sparse.csc_matrix
#         Matrix so that S * w = sum(W) for w = squareform(W).
#     St : scipy.sparse.csc_matrix
#         The adjoint of S (S transpose).
        
#     The output is consistent with the MATLAB function:
#     [S, St] = sum_squareform(n, mask)
#     """
#     if mask is None:
#         mask_given = False
#     else:
#         mask_given = True

#     if mask_given:
#         # Check mask length
#         if len(mask) != n*(n-1)//2:
#             raise ValueError('mask size must be n(n-1)/2')
        
#         # Find nonzero indices in mask
#         # MATLAB code tries to handle both column vectors and row vectors
#         # In Python, let's just do a generic approach:
#         mask = np.asarray(mask).flatten()
#         ind_vec = np.nonzero(mask)[0]  # 0-based indices of nonzero
        
#         ncols = len(ind_vec)

#         I = np.zeros(ncols, dtype=np.int64)
#         J = np.zeros(ncols, dtype=np.int64)

#         curr_row = 1
#         offset = 0
#         # length of current row of matrix, counting from after the diagonal
#         length = n - 1
#         for ii in range(ncols):
#             ind_vec_i = ind_vec[ii] + 1  # switch to 1-based for logic
#             # Move down rows until we find the correct row
#             while ind_vec_i > (length + offset):
#                 offset += length
#                 length -= 1
#                 curr_row += 1
#             # curr_row, J position in MATLAB indexing
#             I[ii] = curr_row
#             # Convert the position within the row into an absolute column index
#             # J(ii) = ind_vec_i - offset + (n - length)
#             J[ii] = ind_vec_i - offset + (n - length)

#     else:
#         # No mask given
#         ncols = (n-1)*n//2
#         I = np.zeros(ncols, dtype=np.int64)
#         J = np.zeros(ncols, dtype=np.int64)

#         # Fill I
#         k = 0
#         for i in range(2, n+1):
#             end_idx = k+(n - i + 1)
#             # I(k : k+(n-i)) = i:n in MATLAB (1-based)
#             # That means for each i, we fill a block of size (n-i+1) with values from i to n.
#             I[k:end_idx] = np.arange(i, n+1)
#             k = end_idx

#         # Fill J
#         k = 0
#         for i in range(2, n+1):
#             end_idx = k+(n - i + 1)
#             # J(k : k+(n-i)) = i-1 in MATLAB
#             # That means for each block, we fill it with i-1 repeated.
#             J[k:end_idx] = (i - 1)
#             k = end_idx

#     # Convert from 1-based to 0-based indexing
#     I -= 1
#     J -= 1

#     # Construct St
#     # St in MATLAB: sparse([1:ncols, 1:ncols],[I, J],1,ncols,n)
#     # means each row i has two ones: one at column I(i), one at column J(i).
#     # In Python (0-based), rows = 0..ncols-1
#     row_indices = np.repeat(np.arange(ncols), 2)
#     col_indices = np.hstack((I, J))
#     data = np.ones(2*ncols, dtype=np.float64)

#     St = coo_matrix((data, (row_indices, col_indices)), shape=(ncols, n)).tocsc()
#     S = St.transpose().tocsc()

#     return S, St
